#Clustering Neighborhoods in Toronto

# Introduction
 This notebook presents an exploration of the Neighborhoods in Toronto, Canada, employing Foursqaure API for data gathering and k-means machine learning algorithm for clustering. Our source data is scrapped from Wikipedia with the aid of the BeautifulSoup library. The Foursqaure API is engaged to obtain the various venues and respective categories associated with the Neighborhoods. The folium library is deployed to present apt visualizations of our Neighborhood clusters following k-means algorithm implementation.

 The Presentation is made in three parts:

[PART ONE: Web Scrapping](#PART-ONE:-Web-Scrapping)

[PART TWO: Exploration](#PART-TWO:-Exploration)

[PART THREE: Clustering and Visualization](#PART-THREE:-Clustering-and-Visualization)





## PART ONE: Web Scrapping
### Import Libraries

In [75]:
import csv
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#!pip install geocoder


### Get webpage html with BeatifulSoup

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Raw_content = requests.get(url).text
soup = BeautifulSoup(Raw_content, 'lxml')
a=soup.find('table', class_='wikitable sortable')

### Extract rows from webpage table into empty Dataframe

In [3]:
table_row = a.find_all('tr')
data = pd.DataFrame()

# Loop through the rows to extract table data
for tr in a.find_all('tr'): 
    td = tr.find_all('td')
    row = [i.text for i in td]
    data = data.append([row])
    
# Specify Dataframe column names
data.columns = ['Post Code', 'Borough', 'Neighborhood']
data = data[1:].reset_index(drop=True)
data.head()

,Post Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### Clean up  Dataframe

In [4]:
columns = data.columns.to_list()

# Remove the undesired '\n' from Dataframe elements
for column in columns:  
    data[column] = data[column].apply(lambda x: x.rstrip('\n'))
    
data.head()

# Drop rows having Borough 'Not assigned'
data.drop(data[data['Borough']=='Not assigned'].index, axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)
data.head() 
    

,Post Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Final Dataframe dimension

In [5]:
print('The number of rows of my Dataframe is {}'.format(data.shape[0]))

The number of rows of my Dataframe is 103


## PART TWO: Exploration

### Download Geospatial Data

In [6]:
import wget
url_coord = 'http://cocl.us/Geospatial_data'
wget.download(url_coord)

100% [................................................................................] 2891 / 2891

'Geospatial_Coordinates (1).csv'

### Read Geospatial Data into pandas Dataframe

In [9]:
RawGeo = pd.read_csv('Geospatial_Coordinates.csv')
RawGeo.rename(columns={'Postal Code':'Post Code'}, inplace=True)
RawGeo.sort_values('Post Code', ascending = True, axis=0, inplace = True)
RawGeo.head()

,Post Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Filter Geospatial Dataframe (RawGeo) based on the primary Dataframe (data) 'Post Code' column content

In [10]:
RawGeo = RawGeo[RawGeo['Post Code'].isin(data['Post Code'])]

### Merge both Dataframes into Geo_Cord

In [11]:
Geo_Cord = pd.merge(RawGeo, data)
Geo_Cord.head()

,Post Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


### Re-order the columns 

In [12]:
filtered_columns = [Geo_Cord.columns[0]] + list(Geo_Cord.columns[3:5]) + list(Geo_Cord.columns[1:3])
filtered_columns

Geo_Cord = Geo_Cord[filtered_columns]
Geo_Cord.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## PART THREE: Clustering and Visualization

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy
from geopy.geocoders import Nominatim

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Group Neighborhoods in Toronto Boroughs

In [39]:
toronto_boroughs=[]
toronto_boroughs = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
Geo_Toronto = Geo_Cord[Geo_Cord['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
Geo_Toronto = Geo_Toronto.iloc[:,1:]
Geo_Toronto.sort_values('Borough', ascending=True, inplace=True, axis=0)
Geo_Toronto = Geo_Toronto.reset_index(drop=True)
Geo_Toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,Central Toronto,Forest Hill North & West,43.696948,-79.411307
2,Central Toronto,Lawrence Park,43.728020,-79.388790
3,Central Toronto,Davisville North,43.712751,-79.390197
4,Central Toronto,North Toronto West,43.715383,-79.405678


### Get Toronto Geo-coordinates

In [40]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Map of Toronto with Neighborhoods superimposed

In [41]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Geo_Toronto['Latitude'], Geo_Toronto['Longitude'], Geo_Toronto['Borough'], Geo_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Exploring all Neighborhoods in Toronto

In [181]:
# Foursquare Credentials

CLIENT_ID = '' # my Foursquare ID
CLIENT_SECRET = '' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [47]:
# Function to implement exploration of all Toronto Neighborhoods

LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
# Obtain Dataframe having the venues explored around Toronto

Toronto_Venues = getNearbyVenues(names=Geo_Toronto['Neighborhood'],
                                   latitudes=Geo_Toronto['Latitude'],
                                   longitudes=Geo_Toronto['Longitude']
                                  )


The Annex, North Midtown, Yorkville
Forest Hill North & West
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Harbourfront East, Union Station, Toronto Islands
University of Toronto, Harbord
Commerce Court, Victoria Hotel
Toronto Dominion Centre, Design Exchange
Queen's Park, Ontario Provincial Government
Richmond, Adelaide, King
Central Bay Street
Regent Park, Harbourfront
St. James Town
Garden District, Ryerson
Kensington Market, Chinatown, Grange Park
Church and Wellesley
St. James Town, Cabbagetown
Rosedale
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Berczy Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
The Beaches
India Bazaar, The Beaches West
The Danforth West, Riverdale
Studio District
Business reply mail Processing Centre
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockt

In [49]:
Toronto_Venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
1,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
2,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
4,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Krispy Kreme Doughnut Cafe,43.674732,-79.407730,Donut Shop


In [50]:
# Dimension of Dataframe
Toronto_Venues.shape

(1614, 7)

In [53]:
# Number of Venues reported per Neighborhood
Toronto_Venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                    57
Brockton, Parkdale Village, Exhibition Place                                                                   22
Business reply mail Processing Centre                                                                          17
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             64
Christie                                                                                                       17
Church and Wellesley                                                                                           75
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

In [64]:
col = list(Toronto_onehot.columns)
col.index('Neighborhood')

162

### Analyzing the Neighborhoods for Venue Category Distribution

In [72]:
# one hot encoding
Toronto_onehot=[]
Toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

## add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_Venues['Neighborhood'] 

## move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[162]] + list(Toronto_onehot.columns[0:162]) + list(Toronto_onehot.columns[163:235])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.000000,0.017544,0.0000,0.0,0.0000,0.0,0.017544,0.000000,0.00000,0.0,0.00000,0.000000,0.0,0.017544,0.035088,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.000000,0.017544,0.052632,0.070175,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.017544,0.0,0.017544,0.0,0.0,0.0,0.0,0.00000,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.0,0.0,0.000000,0.017544,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.000000,

### Top ten venues for each Neighborhood

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Bakery,Beer Bar,Restaurant,Beach,Fish Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,Business reply mail Processing Centre,Smoke Shop,Auto Workshop,Skate Park,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Bar,Harbor / Marina,Boutique,Rental Car Location
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Ice Cream Shop,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant


### Cluster Neighborhoods and Visualize (using k-means clustering algorithm)

In [77]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1) # drop categorical variable

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Geo_Toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Café,Sandwich Place,Pharmacy,Middle Eastern Restaurant,Pub,BBQ Joint,History Museum,Burger Joint,Pizza Place
1,Central Toronto,Forest Hill North & West,43.696948,-79.411307,4,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
2,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
3,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Breakfast Spot,Hotel,Sandwich Place,Food & Drink Shop,Department Store,Park,Eastern European Restaurant,Donut Shop,Dessert Shop
4,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Gift Shop


Visualize Toronto Neighborhood Clusters

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [163]:
# Filter Dataframe to show individual cluster labels 
Cluster_0 = Toronto_merged[Toronto_merged['Cluster Labels'].isin([0])]
Cluster_1 = Toronto_merged[Toronto_merged['Cluster Labels'].isin([1])]
Cluster_2 = Toronto_merged[Toronto_merged['Cluster Labels'].isin([2])]
Cluster_3 = Toronto_merged[Toronto_merged['Cluster Labels'].isin([3])]
Cluster_4 = Toronto_merged[Toronto_merged['Cluster Labels'].isin([4])]  

Create Dataframe to display the three most common venues in each of the clusters

In [248]:
Cluster_Analysis = pd.DataFrame()

Labels = [Cluster_0, Cluster_1, Cluster_2, Cluster_3, Cluster_4]
columns = list(Venue_analysis.columns)
row=[]

for Label in Labels:
    Venue_analysis = Label.loc[:,['1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue']]
    row=[]
    for col in columns:
        row.append(Venue_analysis[col].value_counts().to_frame().index[0])
    Cluster_Analysis = Cluster_Analysis.append([row])
        
    
Cluster_Analysis.columns=columns
Cluster_Analysis['Cluster Labels']=[0,1,2,3,4]
Reorder_column = [Cluster_Analysis.columns[-1]] + list(Cluster_Analysis.columns[:-1])
Cluster_Analysis = Cluster_Analysis[Reorder_column]
Cluster_Analysis.set_index('Cluster Labels')
    

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
Cluster Labels,,,
0,Coffee Shop,Café,Café
1,Park,Playground,Trail
2,Park,Swim School,Bus Line
3,Health & Beauty Service,Garden,Yoga Studio
4,Park,Jewelry Store,Trail
